In [64]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline
from datetime import datetime
import time 
pd.options.display.max_columns = None
pd.set_option('display.max_rows', 1000)
import scipy.stats
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from catboost import CatBoostClassifier
from sklearn.linear_model import BayesianRidge, HuberRegressor, Ridge, OrthogonalMatchingPursuit
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold, cross_val_score
import optuna
#from xgboost import XGBRegressor
#from pycaret.regression import setup, compare_models, models
from scipy import stats
from scipy.stats import boxcox
from scipy.special import boxcox1p
from sklearn import linear_model
from sklearn.linear_model import LinearRegression 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [65]:
df_orders = pd.read_csv('data/Orders.csv',index_col=0)

In [66]:
df_returns = pd.read_csv('data/Returns.csv',index_col=0)

In [67]:
df_orders.head()

,Order.ID,Order.Date,Ship.Date,Ship.Mode,Customer.ID,Customer.Name,Segment,Postal.Code,City,State,Country,Region,Market,Product.ID,Category,Sub.Category,Product.Name,Sales,Quantity,Discount,Profit,Shipping.Cost,Order.Priority
Row.ID,,,,,,,,,,,,,,,,,,,,,,,
40098,CA-2014-AB10015140-41954,11/11/14,11/13/14,First Class,AB-100151402,Aaron Bergman,Consumer,73120.0,Oklahoma City,Oklahoma,United States,Central US,USCA,TEC-PH-5816,Technology,Phones,Samsung Convoy 3,$221.98,2,0.0,$62.15,40.77,High
26341,IN-2014-JR162107-41675,2/5/14,2/7/14,Second Class,JR-162107,Justin Ritter,Corporate,NaN,Wollongong,New South Wales,Australia,Oceania,Asia Pacific,FUR-CH-5379,Furniture,Chairs,"Novimex Executive Leather Armchair, Black","$3,709.40",9,0.1,-$288.77,923.63,Critical
25330,IN-2014-CR127307-41929,10/17/14,10/18/14,First Class,CR-127307,Craig Reiter,Consumer,NaN,Brisbane,Queensland,Australia,Oceania,Asia Pacific,TEC-PH-5356,Technology,Phones,"Nokia Smart Phone, with Caller ID","$5,175.17",9,0.1,$919.97,915.49,Medium
13524,ES-2014-KM1637548-41667,1/28/14,1/30/14,First Class,KM-1637548,Katherine Murray,Home Office,NaN,Berlin,Berlin,Germany,Western Europe,Europe,TEC-PH-5267,Technology,Phones,"Motorola Smart Phone, Cordless","$2,892.51",5,0.1,-$96.54,910.16,Medium
47221,SG-2014-RH9495111-41948,11/5/14,11/6/14,Same Day,RH-9495111,Rick Hansen,Consumer,NaN,Dakar,Dakar,Senegal,Western Africa,Africa,TEC-CO-6011,Technology,Copiers,"Sharp Wireless Fax, High-Speed","$2,832.96",8,0.0,$311.52,903.04,Critical


In [68]:
df_returns.head()

,Order ID,Region
Returned,,
Yes,CA-2012-SA20830140-41210,Central US
Yes,IN-2012-PB19210127-41259,Eastern Asia
Yes,CA-2012-SC20095140-41174,Central US
Yes,IN-2015-JH158207-42140,Oceania
Yes,IN-2014-LC168857-41747,Oceania


In [69]:
df_orders.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51290 entries, 40098 to 6147
Data columns (total 23 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Order.ID        51290 non-null  object 
 1   Order.Date      51290 non-null  object 
 2   Ship.Date       51290 non-null  object 
 3   Ship.Mode       51290 non-null  object 
 4   Customer.ID     51290 non-null  object 
 5   Customer.Name   51290 non-null  object 
 6   Segment         51290 non-null  object 
 7   Postal.Code     9994 non-null   float64
 8   City            51290 non-null  object 
 9   State           51290 non-null  object 
 10  Country         51290 non-null  object 
 11  Region          51290 non-null  object 
 12  Market          51290 non-null  object 
 13  Product.ID      51290 non-null  object 
 14  Category        51290 non-null  object 
 15  Sub.Category    51290 non-null  object 
 16  Product.Name    51290 non-null  object 
 17  Sales           51290 non-nu

In [70]:
df_orders.columns

Index(['Order.ID', 'Order.Date', 'Ship.Date', 'Ship.Mode', 'Customer.ID',
       'Customer.Name', 'Segment', 'Postal.Code', 'City', 'State', 'Country',
       'Region', 'Market', 'Product.ID', 'Category', 'Sub.Category',
       'Product.Name', 'Sales', 'Quantity', 'Discount', 'Profit',
       'Shipping.Cost', 'Order.Priority'],
      dtype='object')

In [71]:
df_orders['Profit'] = df_orders['Profit'].str.replace('$', '')

<ipython-input-71-2c5c2ad100b1>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df_orders['Profit'] = df_orders['Profit'].str.replace('$', '')


In [72]:
df_orders['Profit'] = df_orders['Profit'].str.replace(',', '')

In [73]:
df_orders.Profit=df_orders.Profit.astype('float64')

In [74]:
df_orders.Sales

Row.ID
40098      $221.98
26341    $3,709.40
25330    $5,175.17
13524    $2,892.51
47221    $2,832.96
           ...    
29002       $65.10
34337       $16.72
31315        $8.56
9596        $13.44
6147        $61.38
Name: Sales, Length: 51290, dtype: object

In [75]:
df_orders['Sales'] = df_orders['Sales'].str.replace('$', '')
df_orders['Sales'] = df_orders['Sales'].str.replace(',', '')
df_orders.Sales=df_orders.Sales.astype('float64')

<ipython-input-75-f31c380abf8f>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df_orders['Sales'] = df_orders['Sales'].str.replace('$', '')


In [76]:
df_orders.Sales

Row.ID
40098     221.98
26341    3709.40
25330    5175.17
13524    2892.51
47221    2832.96
          ...   
29002      65.10
34337      16.72
31315       8.56
9596       13.44
6147       61.38
Name: Sales, Length: 51290, dtype: float64

In [77]:
df_orders.Profit

Row.ID
40098     62.15
26341   -288.77
25330    919.97
13524    -96.54
47221    311.52
          ...  
29002      4.50
34337      3.34
31315      2.48
9596       2.40
6147       1.80
Name: Profit, Length: 51290, dtype: float64

# Problem 2: Inventory Management

In [78]:
df_orders["Order.Date"]=pd.to_datetime(df_orders["Order.Date"])

In [79]:
df_orders["Ship.Date"]=pd.to_datetime(df_orders["Ship.Date"])

In [80]:
df_orders["Ouarters"] = pd.PeriodIndex(df_orders["Order.Date"], freq='Q')

In [83]:
df_orders.head()

,Order.ID,Order.Date,Ship.Date,Ship.Mode,Customer.ID,Customer.Name,Segment,Postal.Code,City,State,Country,Region,Market,Product.ID,Category,Sub.Category,Product.Name,Sales,Quantity,Discount,Profit,Shipping.Cost,Order.Priority,Ouarters
Row.ID,,,,,,,,,,,,,,,,,,,,,,,,
40098,CA-2014-AB10015140-41954,2014-11-11,2014-11-13,First Class,AB-100151402,Aaron Bergman,Consumer,73120.0,Oklahoma City,Oklahoma,United States,Central US,USCA,TEC-PH-5816,Technology,Phones,Samsung Convoy 3,221.98,2,0.0,62.15,40.77,High,2014Q4
26341,IN-2014-JR162107-41675,2014-02-05,2014-02-07,Second Class,JR-162107,Justin Ritter,Corporate,NaN,Wollongong,New South Wales,Australia,Oceania,Asia Pacific,FUR-CH-5379,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",3709.40,9,0.1,-288.77,923.63,Critical,2014Q1
25330,IN-2014-CR127307-41929,2014-10-17,2014-10-18,First Class,CR-127307,Craig Reiter,Consumer,NaN,Brisbane,Queensland,Australia,Oceania,Asia Pacific,TEC-PH-5356,Technology,Phones,"Nokia Smart Phone, with Caller ID",5175.17,9,0.1,919.97,915.49,Medium,2014Q4
13524,ES-2014-KM1637548-41667,2014-01-28,2014-01-30,First Class,KM-1637548,Katherine Murray,Home Office,NaN,Berlin,Berlin,Germany,Western Europe,Europe,TEC-PH-5267,Technology,Phones,"Motorola Smart Phone, Cordless",2892.51,5,0.1,-96.54,910.16,Medium,2014Q1
47221,SG-2014-RH9495111-41948,2014-11-05,2014-11-06,Same Day,RH-9495111,Rick Hansen,Consumer,NaN,Dakar,Dakar,Senegal,Western Africa,Africa,TEC-CO-6011,Technology,Copiers,"Sharp Wireless Fax, High-Speed",2832.96,8,0.0,311.52,903.04,Critical,2014Q4


In [86]:
df_orders.Ouarters.apply(lambda x: str(x)[-1])

Row.ID
40098    4
26341    1
25330    4
13524    1
47221    4
        ..
29002    2
34337    2
31315    3
9596     1
6147     2
Name: Ouarters, Length: 51290, dtype: object

In [87]:
df_orders.Ouarters=df_orders.Ouarters.apply(lambda x: str(x)[-1])

In [88]:
df_orders.head()

,Order.ID,Order.Date,Ship.Date,Ship.Mode,Customer.ID,Customer.Name,Segment,Postal.Code,City,State,Country,Region,Market,Product.ID,Category,Sub.Category,Product.Name,Sales,Quantity,Discount,Profit,Shipping.Cost,Order.Priority,Ouarters
Row.ID,,,,,,,,,,,,,,,,,,,,,,,,
40098,CA-2014-AB10015140-41954,2014-11-11,2014-11-13,First Class,AB-100151402,Aaron Bergman,Consumer,73120.0,Oklahoma City,Oklahoma,United States,Central US,USCA,TEC-PH-5816,Technology,Phones,Samsung Convoy 3,221.98,2,0.0,62.15,40.77,High,4
26341,IN-2014-JR162107-41675,2014-02-05,2014-02-07,Second Class,JR-162107,Justin Ritter,Corporate,NaN,Wollongong,New South Wales,Australia,Oceania,Asia Pacific,FUR-CH-5379,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",3709.40,9,0.1,-288.77,923.63,Critical,1
25330,IN-2014-CR127307-41929,2014-10-17,2014-10-18,First Class,CR-127307,Craig Reiter,Consumer,NaN,Brisbane,Queensland,Australia,Oceania,Asia Pacific,TEC-PH-5356,Technology,Phones,"Nokia Smart Phone, with Caller ID",5175.17,9,0.1,919.97,915.49,Medium,4
13524,ES-2014-KM1637548-41667,2014-01-28,2014-01-30,First Class,KM-1637548,Katherine Murray,Home Office,NaN,Berlin,Berlin,Germany,Western Europe,Europe,TEC-PH-5267,Technology,Phones,"Motorola Smart Phone, Cordless",2892.51,5,0.1,-96.54,910.16,Medium,1
47221,SG-2014-RH9495111-41948,2014-11-05,2014-11-06,Same Day,RH-9495111,Rick Hansen,Consumer,NaN,Dakar,Dakar,Senegal,Western Africa,Africa,TEC-CO-6011,Technology,Copiers,"Sharp Wireless Fax, High-Speed",2832.96,8,0.0,311.52,903.04,Critical,4


In [90]:
df_orders.groupby("Ouarters").Quantity.sum()
#Quantity vs order.date

Ouarters
1    26691
2    41671
3    49121
4    60829
Name: Quantity, dtype: int64

Is there any seasonal trend of inventory in the company?
Q4 highest, Q1 lowest

Is the seasonal trend the same for different categories?

In [91]:
df_orders.groupby("Category","Ouarters").Quantity.sum()

ValueError: No axis named Ouarters for object type DataFrame